In [1]:
import numpy as np
import pylab as pl
import riemannsolver as riemann

# Constants and Configuration
GAMMA = 5. / 3.
TIMESTEP = 0.001
NUMSTEPS = 200
NUMCELLS = 100

In [2]:
class Cell:
    def __init__(self, midpoint, volume):
        self.midpoint = midpoint
        self.volume = volume
        self.mass = 0.
        self.momentum = 0.
        self.energy = 0.
        self.density = 0.
        self.velocity = 0.
        self.pressure = 0.
        self.right_ngb = None

def initialize_cells(num_cells):
    cells = []
    for i in range(num_cells):
        cell = Cell((i + 0.5) / num_cells, 1. / num_cells)
        if i < num_cells / 2:
            cell.mass = cell.volume
            cell.energy = cell.volume / (GAMMA - 1.)
        else:
            cell.mass = 0.125 * cell.volume
            cell.energy = 0.1 * cell.volume / (GAMMA - 1.)
        if cells:
            cells[-1].right_ngb = cell
        cells.append(cell)
    return cells

def update_primitive_variables(cells):
    for cell in cells:
        cell.density = cell.mass / cell.volume
        cell.velocity = cell.momentum / cell.mass
        cell.pressure = (GAMMA - 1.) * (cell.energy / cell.volume - 0.5 * cell.density * cell.velocity**2)


def solve_riemann_problems(cells, solver):
    for cell in cells[:-1]:  # Skip the last cell
        solve_flux_exchange(cell, cell.right_ngb, solver)


def solve_flux_exchange(cell, cell_right, solver):
    densityL, velocityL, pressureL = cell.density, cell.velocity, cell.pressure
    densityR, velocityR, pressureR = cell_right.density, cell_right.velocity, cell_right.pressure

    # Solve the Riemann problem
    densitysol, velocitysol, pressuresol, _ = solver.solve(densityL, velocityL, pressureL, densityR, velocityR, pressureR)

    # Compute the fluxes
    flux_mass = densitysol * velocitysol
    flux_momentum = densitysol * velocitysol ** 2 + pressuresol
    flux_energy = (pressuresol * GAMMA / (GAMMA - 1) + 0.5 * densitysol * velocitysol ** 2) * velocitysol

    # Update the states based on fluxes
    A = 1.0  # Assuming surface area = 1 for 1D
    cell.mass -= flux_mass * A * TIMESTEP
    cell.momentum -= flux_momentum * A * TIMESTEP
    cell.energy -= flux_energy * A * TIMESTEP

    cell_right.mass += flux_mass * A * TIMESTEP
    cell_right.momentum += flux_momentum * A * TIMESTEP
    cell_right.energy += flux_energy * A * TIMESTEP


def apply_boundary_conditions(cells, solver):
    # Reflective boundary at the left edge
    cell_first = cells[0]
    densityL, velocityL, pressureL = -cell_first.velocity, cell_first.density, cell_first.pressure
    
    # Since it's reflective, the right state is a mirror of the left with a negated velocity
    densityR, velocityR, pressureR = cell_first.density, -cell_first.velocity, cell_first.pressure

    # Solve the Riemann problem at the boundary
    densitysol, velocitysol, pressuresol, _ = solver.solve(densityL, velocityL, pressureL, densityR, velocityR, pressureR)

    # Compute and apply the flux at the boundary
    flux_mass = densitysol * velocitysol
    flux_momentum = densitysol * velocitysol ** 2 + pressuresol
    flux_energy = (pressuresol * GAMMA / (GAMMA - 1) + 0.5 * densitysol * velocitysol ** 2) * velocitysol

    A = 1.0  # Surface area for 1D
    cell_first.mass += flux_mass * A * TIMESTEP
    cell_first.momentum += flux_momentum * A * TIMESTEP
    cell_first.energy += flux_energy * A * TIMESTEP

    # Reflective boundary at the right edge (optional based on problem setup)
    # Similar logic to above can be applied for the right-most cell, if needed



def plot_solution(cells, timestep, num_steps):
    # Prepare the data: extract cell midpoints and densities
    midpoints = [cell.midpoint for cell in cells]
    densities = [cell.density for cell in cells]
    
    # Optional: Compute reference solution for comparison
    # xref = np.linspace(0, 1, 1000) # Example, adjust as necessary
    # rhoref = [reference_solution(x, timestep*num_steps) for x in xref] # Define this function based on the analytical solution

    # Create the plot
    pl.figure(figsize=(10, 6))  # Optional: Adjust figure size
    # pl.plot(xref, rhoref, "r-", label="Reference Solution")  # Uncomment if reference solution is available
    pl.plot(midpoints, densities, "k.", label="Simulated Density")
    
    # Customize the plot
    pl.ylim(0, max(densities) * 1.1)  # Adjust Y-axis limits to better fit the data
    pl.xlabel("Position")
    pl.ylabel("Density")
    pl.title("Density Profile at Final Time")
    pl.legend()
    pl.grid(True)  # Optional: Add grid for easier visualization
    
    # Save and/or display the plot
    pl.tight_layout()  # Adjust subplot parameters to fit into the figure area
    pl.savefig("sodshock_density_profile.png")  # Save the plot as a PNG file
    pl.show()  # Display the plot interactively



def main():
    solver = riemann.RiemannSolver(GAMMA)
    cells = initialize_cells(NUMCELLS)
    for _ in range(NUMSTEPS):
        update_primitive_variables(cells)
        solve_riemann_problems(cells, solver)
        apply_boundary_conditions(cells, solver)
    plot_solution(cells, solver, TIMESTEP, NUMSTEPS)

if __name__ == "__main__":
    main()


ZeroDivisionError: float division by zero